<a href="https://colab.research.google.com/github/macorony/Bioinformatic_analysis/blob/main/Algorithms/Bayesian_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import scipy

In [11]:
import numpy as np
import pandas as pd
class BayesianSelection:
  def __init__(self, count_table, design_matrix=None, sgrna_efficiency=None):
    self.count_table = count_table
    self.design_matrix = design_matrix
    self.sgrna_efficiency = sgrna_efficiency

    self.n_sgrnas = len(count_table)
    self.genes = np.unique(count_table['gene'])
    self.n_genes = len(self.genes)

    self.normalized_counts = None
    self.fold_changes = None
    self.gene_guide_map = None

  def normalize_counts(self, count_table):
    control_sf = np.median(count_table['control'])/count_table['control']
    treatment_sf = np.median(count_table['treatment'])/count_table['treatment']
    normalized = pd.DataFrame(
        {'control': count_table['control']*control_sf,
         'treatment': count_table['treatment']*treatment_sf}
        )
    return normalized

  def group_sgrna_by_gene(self):
    gene_guide_map = {}
    for gene in self.genes:
      gene_guide_map[gene] = np.where(self.count_table['gene'] == gene)
    return gene_guide_map

  def initialize_priors(self):
    '''
    Initialize prior distributions for Bayesian analysis.
    '''
    # 1. gene effect priors(normal distribution)
    self.gene_priors = {
        'mean': np.zeros(self.n_genes),
        'variance': np.ones(self.n_genes)
    }
    # 2.sgRNA efficiency priors
    if self.sgrna_efficiency is None:
      self.sgrna_priors = {
          'mean': np.array(list(self.sgrna_efficiency.values())),
          'variance': 0.1 * np.ones(self.n_sgrnas)
      }
    else:
      self.sgrna_priors = {
          'mean': np.ones(self.n_sgrnas),
          'variance':np.ones(self.n_sgrnas)
          }
    # 3. dispersion priors (Gamma distribution)
    self.dispersion_priors = {
        'shape': np.ones(self.n_genes),
        'scale': np.ones(self.n_genes)
        }
    # 4. size factor priors (if needed for multiple conditions)
    if self.design_matrix is not None:
      n_conditions = self.design_matrix.shape[1]
      self.size_factor_priors = {
          'mean': np.ones(n_conditions),
          'variance': 0.1 * np.ones(n_conditions)
          }

  def construct_model(self):
    """
    Construct the hierarchical model for Bayesian analysis.
    """
    self.model = {
        # likelihood function (Negative Binomial)
        'likelihood': self.contruct_likelihood(),
        # priors?
        'priors': self.initialize_priors(),
        # hyperparameters?
        'hyperparameters': self.set_hyperparameters()
    }

  def estimate_parameters(self, n_iterations=1000, burn_in=100):
    """
    Estimate the parameters using EM algorithm.
    """
    self.mcmc_samples = {
        'gene_effect': np.zeros((n_iterations, self.n_genes)),
        'sgrna_efficiency': np.zeros((n_iterations, self.n_sgrnas)),
        'dispersion': np.zeros((n_iterations, self.n_genes))
    }
    # run gibbs sampling
    for iter in range(n_iterations):
      self.update_gene_effect()
      self.update_sgrna_efficiency()
      self.update_dispersion()
      # store sample (after burn-in)
      if iter >= burn_in:
        self.store_samples(iter)
      # monitor convergence
      if iter % 100 == 0:
        self.check_convergence(iter)

  def contruct_likelihood(self):
    """
    contruct negative binomial likelihood function
    """
    def negative_binomial_likelihood(count, mean, dispersion):
      # log likelihood of negative binomial distribution
      r = 1/dispersion
      p = r/(r+mean)
      l1 = scipy.stats.nbinom.logpmf(count, r, p)
      return l1
    return negative_binomial_likelihood

  def update_gene_effect(self):
    """
    Update gene effects using Gibbs sampling.
    """
    for gene_idx, gene in enumerate(self.n_genes):
      # get sgRNA for the gene
      gene_guides = self.gene_guide_map[self.genes[gene_idx]]['guide_index']
      # calculate statistics
      sgrna_data = self.fold_changes.loc[gene_guides, 'log2fc']
      sgrna_vars = 1/(self.sgrna_prior['variance'][gene_guides])
      # posterior parameters
      posterior_var = 1/(1/self.gene_prior['variance'][gene_idx] + np.sum(sgrna_vars))
      posterior_mean = posterior_var * (self.gene_prior['mean'][gene_idx]/self.gene_prior['variance'][gene_idx] + np.sum(sgrna_data*sgrna_vars))
      # sample new effect
      self.gene_effects[gene_idx] = np.random.normal(posterior_mean, np.sqrt(posterior_var))
  def update_sgrna_efficiency(self):
    """
    Update sgRNA efficiencies.
    """
    for sgrna_idx, sgrna in enumerate(self.n_sgrnas):
      # calculate statistics
      gene = self.count_table.loc[sgrna_idx, 'gene']
      gene_idx = np.where(self.genes == gene)[0][0]
      # calculate posterior
      data_contribution = self.fold_changes.loc[sgrna_idx, 'log2fc']
      prior_contribution = self.sgrna_prior['mean'][gene_idx]

      posterior_var = 1/(
          1/self.sgrna_priors['variance'][sgrna_idx] +
          1/self.gene_effects[gene_idx]**2
      )
      posterior_mean = posterior_var * (
          data_contribution/self.fold_changes.loc[sgrna_idx, 'variance'] +
          prior_contribution/self.gene_effects[gene_idx]**2
      )

      # sample the efficiency
      self.sgrna_efficiency[sgrna] = np.random.normal(posterior_mean, np.sqrt(posterior_var))

  def update_dispersion(self):
    """
    Update dispersion parameters.
    """
    for gene_idx, in enumerate(self.genes):
      gene_guides = self.gene_guide_map[self.genes[gene_idx]]['guide_index']
      # calculate statistics
      shape = self.dispersion_priors['shape'][gene_idx] + len(gene_guides)/2
      rate = self.dispersion_priors['rate'][gene_idx] + \
              np.sum((self.fold_changes.loc[gene_guides, 'log2fc'] -
                    self.gene_effects[gene_idx])**2)/2

      # Sample new dispersion
      self.dispersion[gene_idx] = np.random.gamma(shape, 1/rate)







In [ ]:
count_data = pd.DataFrame(
    {'sgrna': ['sgrna1', 'sgrna2', 'sgrna3', 'sgrna4'],
     'gene': ['geneA', 'geneA', 'geneB', 'geneB'],
     'control': [100, 200, 200, 250],
     'treatment': [50, 150, 200, 400]
     })

In [ ]:
sgrna_efficiency = {'sgrna1':0, 'sgrna2':0, 'sgrna3':0, 'sgrna4':0}

In [ ]:
np.array(list(sgrna_efficiency.values()))

array([0, 0, 0, 0])

In [ ]:
bayes_selector = BayesianSelection(count_data, sgrna_efficiency=sgrna_efficiency)

In [ ]:
bayes_selector.initialize_priors()

In [ ]:
print(bayes_selector.gene_priors)
print(bayes_selector.sgrna_priors)
print(bayes_selector.dispersion_priors)

{'mean': array([0., 0.]), 'variance': array([1., 1.])}
{'mean': array([1., 1., 1., 1.]), 'variance': array([1., 1., 1., 1.])}
{'shape': array([1., 1.]), 'scale': array([1., 1.])}


In [4]:
def example_update():
    # Example data
    prior_mean = 0
    prior_var = 1
    guide_data = np.array([0.5, 0.7, 0.3])
    guide_vars = np.array([0.1, 0.1, 0.1])

    # Calculate posterior
    posterior_var = 1 / (1/prior_var + np.sum(1/guide_vars))
    posterior_mean = posterior_var * (
        np.sum(guide_data/guide_vars) +
        prior_mean/prior_var
    )

    # Sample
    new_effect = np.random.normal(posterior_mean, np.sqrt(posterior_var))

    return new_effect

In [8]:
example_update()

0.588096056993475

In [10]:
np.random.normal(1,4)

0.8626135969033462